## Day 23

In [17]:
from collections import defaultdict

def regOrValue(c,reg):
    try:
        v = int(c)
    except:
        v = reg[c]
    return v

def coprocessor(prog,a=0,count=-1,verbose=False):   
    reg = defaultdict(int) 
    reg['a']=a
    i = 0
    imul = 0
    c = 0
    while True:    
        if i>=len(prog):
            break
        cmd = prog[i].split(" ")        
        if cmd[0]=="set":
            reg[cmd[1]]  = regOrValue(cmd[2],reg)
        elif cmd[0]=="sub":
            reg[cmd[1]] -= regOrValue(cmd[2],reg)
        elif cmd[0]=="mul":
            reg[cmd[1]] *= regOrValue(cmd[2],reg)
            imul += 1
        elif cmd[0]=="jnz":
            if regOrValue(cmd[1],reg)!=0:
                i += regOrValue(cmd[2],reg) - 1 # will additionally increment by 1 as for all other commands
        if verbose: # and reg['h']>0:
            #print(reg)
            print(i,end=" - ")
            for k,v in zip(reg.keys(),reg.values()):
                print(k,v,end="\t")
            print()
        if reg['h']>0:
            break
        i += 1
        c += 1
        if count>0 and c>count:
            break
    return imul, reg

In [18]:
with open("data/input23.txt") as f:
    prog = [l.strip("\n") for l in f.readlines()]

In [19]:
imul, reg = coprocessor(prog)
print("Part 1:",imul)

Part 1: 5929


## Part 2

### Scrutininzing the registers

In [21]:
imul, reg = coprocessor(prog,a=1,count=30,verbose=True)
#reg

0 - a 1	b 79	
1 - a 1	b 79	h 0	c 79	
3 - a 1	b 79	h 0	c 79	
4 - a 1	b 7900	h 0	c 79	
5 - a 1	b 107900	h 0	c 79	
6 - a 1	b 107900	h 0	c 107900	
7 - a 1	b 107900	h 0	c 124900	
8 - a 1	b 107900	h 0	c 124900	f 1	
9 - a 1	b 107900	h 0	c 124900	f 1	d 2	
10 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 2	
11 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 2	g 2	
12 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 2	g 4	
13 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 2	g -107896	
15 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 2	g -107896	
16 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 3	g -107896	
17 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 3	g 3	
18 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 3	g -107897	
10 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 3	g -107897	
11 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 3	g 2	
12 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 3	g 6	
13 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 3	g -107894	
15 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 3	g -107894	
16 - a 1	b 107900	h 0	c 124900	f 1	d 2	e 4	g -107894	
17 - a 1	b 107900	h 0	c 124900	f 1	d

In [15]:
for i in range(len(prog)):
    print(i+1," - ",prog[i])

1  -  set b 79
2  -  set c b
3  -  jnz a 2
4  -  jnz 1 5
5  -  mul b 100
6  -  sub b -100000
7  -  set c b
8  -  sub c -17000
9  -  set f 1
10  -  set d 2
11  -  set e 2
12  -  set g d
13  -  mul g e
14  -  sub g b
15  -  jnz g 2
16  -  set f 0
17  -  sub e -1
18  -  set g e
19  -  sub g b
20  -  jnz g -8
21  -  sub d -1
22  -  set g d
23  -  sub g b
24  -  jnz g -13
25  -  jnz f 2
26  -  sub h -1
27  -  set g b
28  -  sub g c
29  -  jnz g 2
30  -  jnz 1 3
31  -  sub b -17
32  -  jnz 1 -23


### Reverse engineer the program

* in order to exit, the program has to reach line 30 `jnz 1 3`
* to get to line 30, `g` = 0 at line 29 (no jump executed at 31)
* given lines 27 and 28, in order to have `g`=0 I need `b`=`c`
* if `g` != 0, programs get to line 31 where `sub b -17` increments `b` by 17
* the first 10 instructions set `b` = 107900 and `c` = 124900, so part of the program is a loop between 107900 and 124900 at increments of 17, stored in `b`.
* line 26 increments `h` (`sub d -1`)
* programs gets to line 26 if `g` = 0 (line 24, that otherwise brings it back to line 11) and `f` = 0 (that otherwise jumps to 27, where, assuming  `g` = 0, programs runs to its end
* `f` = 0 if `g` = 0 at line 15
* `g` = 0 at line 15 if `d`*`e`=`b` (lines 12, 13 and 14)
* `d` and `e` increment by 1 each in a loop (lines 17 and 21) and both `d` and `e` **begins at 2**
* in order to advance further I also need to pass line 20 `jnz g -8` with `g`=0
* in order to satify the above conditions, `b` must be divisible by at least a number greater or equal then 2, thus **`b` must not be prime**

**I think the program counts the non-prime numbers in the sequence between 107900 and 124900 with increments of 17**

In [29]:
h = 0
for b in range(107900,124900+1,17):
    for d in range(2,b):
        if b%d==0:
            h+=1
            break
print(h)

907
